In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('steam-200k.csv', usecols=[0, 1, 2, 3],
                 names=['USER ID', 'GAME TITLE', 'BEHAVIOUR', 'VALUE'])

In [7]:
# import pandas_profiling
# df.profile_report()

In [6]:
df['HOURS'] = df['VALUE']
df.loc[(df['BEHAVIOUR'] == 'purchase'), 'HOURS'] = 0
df = df.sort_values(['USER ID', 'GAME TITLE', 'HOURS'])
df = df.drop_duplicates(['USER ID', 'GAME TITLE'], keep='last').drop(columns=['BEHAVIOUR', 'VALUE'])

In [15]:
# n1 = len(df['USER ID'].unique())
# n2 = len(df['GAME TITLE'].unique())

In [ ]:
# df_train, df_test = train_test_split(df, random_state=42)

In [80]:
user2idx = {user: i for i, user in enumerate(df['USER ID'].unique())}
idx2user = {i: user for user, i in user2idx.items()}

game2idx = {game: i for i, game in enumerate(df['GAME TITLE'].unique())}
idx2game = {i: game for game, i in game2idx.items()}

In [7]:
matrix = df.pivot(index='USER ID', columns='GAME TITLE', values='HOURS')

In [8]:
matrix += 1

In [9]:
conf_matrix = matrix.fillna(0)

In [13]:
# train_conf_matrix, test_conf_matrix = train_test_split(conf_matrix, random_state=42, test_size=0.1)

In [ ]:
pref_matrix = conf_matrix.astype(bool).astype(int)

In [14]:
# train_pref_matrix = train_conf_matrix.astype(bool).astype(int)
# test_pref_matrix = test_conf_matrix.astype(bool).astype(int)

In [11]:
# def next_batch(matrix, batch_size=256):
#     for i in range(0, matrix.shape[0], batch_size):
#         yield matrix[i: i + batch_size]

In [49]:
# def train(matrix, identity, learning_rate=0.01, max_iter=10, rank=10):
#     n1 = matrix.shape[0]
#     n2 = matrix.shape[1]
# #     loss_history = []
#     u = np.random.normal(size=(n1, rank))
#     v = np.random.normal(size=(rank, n2))
#     for k in range(max_iter):
#         for i in range(n1):
#             print(np.dot(u[i], v).shape)
#             print((matrix[i] - np.dot(u[i], v)).shape)
#             print(np.dot(identity[i], matrix[i] - np.dot(u[i], v)).shape)
#             print(np.dot(np.dot(identity[i], matrix[i] - np.dot(u[i], v)), v).shape)
#             u[i] += - learning_rate * (np.dot(np.dot(identity[i], matrix[i] - np.dot(u[i], v)), v) - u[i])
#         for j in range(n2):
#             v[j] += - learning_rate * (np.dot(np.dot(identity.T[j], matrix.T[j] - np.dot(u, v[j])), u) - v[j])
#     return u, v

In [ ]:
def train(matrix, identity, learning_rate=0.01, max_iter=10, rank=2):
    n1 = matrix.shape[0]
    n2 = matrix.shape[1]
    u = np.random.normal(size=(n1, rank))
    v = np.random.normal(size=(n2, rank))
    for k in range(max_iter):
#         print(f'epoch {k}')
        for i in range(n1):
            for j in range(n2):
                s = identity[i][j] * (matrix[i][j] - np.dot(u[i], v[j])) * v[j]
            u[i] += - learning_rate * (s + u[i])
        for j in range(n2):
            for i in range(n1):
                s = identity[i][j] * (matrix[i][j] - np.dot(u[i], v[j])) * u[i]
            v[j] += - learning_rate * (s + v[j])
    return u, v

In [ ]:
u, v = train(conf_matrix.values, pref_matrix.values)

epoch 0


In [93]:
for user_idx in np.random.choice(list(idx2user.keys()), size=4, replace=False):
    user = idx2user[user_idx]
    games = {}
    for game_idx, game in idx2game.items():
        games[game] = np.dot(u[user_idx], v[game_idx])
    games = sorted(games.items(), key= lambda item: item[1], reverse=True)
    top10 = [games[i][0] for i in range(10)]
    print(f'Top 10 Recommended Games for user #{user}: {top10}\n\n')

Top 10 Recommended Games for user #70739775: ['Saviors', 'Crazy Machines 1.5 Inventors Training Camp', '3DMark Cloud Gate benchmark', 'Beyond Divinity', 'Patrician IV Steam Special Edition', 'Ducati World Championship', 'Driftmoon', 'CSGO Player Profiles - Olofmeister', "Alan Wake's American Nightmare", 'The Knobbly Crook Chapter I - The Horse You Sailed In On']


Top 10 Recommended Games for user #169225463: ['Age of Mythology Extended Edition', 'Landmark', 'UberStrike', 'Knights of Honor', 'Crazy Machines Elements', 'I-Fluid', 'Remnants Of Isolation', "Tiny and Big Grandpa's Leftovers", 'The Four Kings Casino and Slots', 'Arma 2 Operation Arrowhead Beta (Obsolete)']


Top 10 Recommended Games for user #60859324: ['Dream Pinball 3D', 'Sonic Adventure DX', 'Deadbreed', 'Nancy Drew Danger By Design', 'Woody Two-Legs Attack of the Zombie Pirates', 'The Guild II', 'Starvoid', 'The Stomping Land', 'TOXIKK', 'PewDiePie Legend of the Brofist']




IndexError: index 11751 is out of bounds for axis 0 with size 11153